# -Shop assist AI-

In [1]:
import pandas as pd
from IPython.display import display, HTML

In [2]:
df = pd.read_csv('laptop_data.csv')
df.head(1)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...


In [3]:
df['Description'][0]

'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.'

## Implementation

In [4]:
import os, json, re, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [5]:
openai.api_key = open('OPENAI_API_Key.txt', 'r').read().strip()
os.environ['OPEN_AI_APIKEY'] = openai.api_key

## Step-1: Initialize conversation

In [6]:
def initialize_conversation():
    delimiter = "####"
    example_user_requirement = {'GPU intensity':'high', 'Display quality':'high', 'Portability':'low', 'Multitasking':'high', 'Processing speed':'high'}
    sys_msg = f"""
    You are an intelligent laptop gadget and your goal is to find the best laptop for the user.
    You need to ask relevant questions and understand the user profile by analize the user response.
    Your final objective is to fill the values for the different keys ('GPU intensity', 'Display quality', 'portability', 'Multitasking', 'Processing speed')
    The key value pairs define the user's profile.
    The python dictionary look like this {{'GPU intensity':'values', 'Display quality':'values', 'portability':'values', 'Multitasking':'values', 'Processing speed':'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', 'high' based on the importance of the product..
    The value for 'budget' should be numerical value extracted from user's response
    The value currently in the dictioanry are only representative values.
    
    {delimiter} Here are some instructions around the values for the different keys. If you do not follow this, you will be heavily penalized.
    - the values for all the keys, except 'budget', should strictly be either 'low', 'medium', 'high' based on the importance of the corresponding keys, as stated by user. 
    - the value for 'budget' should be numerical value extracted from the user's response.
    'Budget' value need to be greater than or equal to the 25000 INR. If user says less than that, please mension that there are no laptop in this range.
    Do not randomly assign values to any of the keys. The values need to be inferred from the user's respone.
    {delimiter}
    
     To fill the dictionary, you need to have the following chain of the thoughts:
     {delimiter} Thought 1:Ask a question to understand the user's profile and requirements. \n
     If their primary use of the laptop is unclear.Ask another question to comprehend their needs.
     You are trying to fill the values of all the keys('GPU intensity', 'Display quality', 'portability', 'Multitasking', 'Processing speed').
     Identify the keys for which you can fill the values confidently using the understandings. \n
     Remember the instructions around the values for the different keys.
     Answer 'Yes' or 'No' to indicate if you understand the requirements and have updated the values for the relevant keys. \n
     If yes, proceed to the next step. Otherwise, perhaps the question to capture their profile. \n{delimiter}
     
     {delimiter} Thought 2:Now you are tryin to fill the values for the rest of the keys which you couldn't in the previous step. \n
     Remember the instructions for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
     Answer 'Yes' or 'No' to indicate if you understood all the values for the keys and are confident about the same.
     If yes, move to the next Thought. If no ask question on the keys whose values you are unsure of. \n
     it is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}
     
     {delimiter} Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
     If you are not confident about any of the values, ask clarifying question.{delimiter}
     
     Follow the above chain of thoughts and only updated python dictionary. \n
     
     
     {delimiter} Here is sample conversation between user and assistent:
     User: "Hi, I am an editor"
     Assistant: "Great! as an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. "
     User: "I primarily work with after effects."
     Assistant: Thank you for providing that information. Working with after effects involves working with graphics, animations and rendering which will require High GPU 
     User: "Yes, sometime I work with a 4k video as well"
     Assitant: "Thank you for providing that information. Processing 4k videos will require a good processor and high GPU. I think we have already determind that"
     User: "Yes, sometime I travel but do not carry my laptop.""
     Assistant: "Could you kindly let me know your budget for the laptop? This will help me to find options that fit your budget.""
     User: "My max budget is 1.5 lakhs inr"
     Assistant: "{example_user_requirement}"
     {delimiter}
     Start with a short welcome message and encourage the user to share their requirements.
    """
    
    
    conversation = [{"role":"system", "content":sys_msg}]
    return conversation

In [7]:
check_conversation = initialize_conversation()
print(check_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent laptop gadget and your goal is to find the best laptop for the user.\n    You need to ask relevant questions and understand the user profile by analize the user response.\n    Your final objective is to fill the values for the different keys (\'GPU intensity\', \'Display quality\', \'portability\', \'Multitasking\', \'Processing speed\')\n    The key value pairs define the user\'s profile.\n    The python dictionary look like this {\'GPU intensity\':\'values\', \'Display quality\':\'values\', \'portability\':\'values\', \'Multitasking\':\'values\', \'Processing speed\':\'values\'}\n    The values for all keys, except \'budget\', should be \'low\', \'medium\', \'high\' based on the importance of the product..\n    The value for \'budget\' should be numerical value extracted from user\'s response\n    The value currently in the dictioanry are only representative values.\n    \n    #### Here are some instructions around the valu

In [8]:
system_msg = check_conversation[0]['content']
print(system_msg)


    You are an intelligent laptop gadget and your goal is to find the best laptop for the user.
    You need to ask relevant questions and understand the user profile by analize the user response.
    Your final objective is to fill the values for the different keys ('GPU intensity', 'Display quality', 'portability', 'Multitasking', 'Processing speed')
    The key value pairs define the user's profile.
    The python dictionary look like this {'GPU intensity':'values', 'Display quality':'values', 'portability':'values', 'Multitasking':'values', 'Processing speed':'values'}
    The values for all keys, except 'budget', should be 'low', 'medium', 'high' based on the importance of the product..
    The value for 'budget' should be numerical value extracted from user's response
    The value currently in the dictioanry are only representative values.
    
    #### Here are some instructions around the values for the different keys. If you do not follow this, you will be heavily penalized.

### Get chat completions

In [9]:
def get_chat_completion(msg):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = msg,
        temperature = 0,
        max_tokens = 900    
    )
    return response.choices[0].message.content

In [10]:
check_introduction = get_chat_completion(check_conversation)
print(check_introduction)

Hello! I'm here to help you find the best laptop that suits your needs. Please share your requirements with me so that I can assist you effectively.


In [ ]:
user_input = "Hi, I am Abhijit. I need a laptop for coding and gaming etc.."

In [12]:
check_conversation.append({"role":"user", "content":user_input})

In [13]:
check_assistent_response = get_chat_completion(check_conversation)
print(check_assistent_response)

Great! It seems like you have a variety of requirements including coding, gaming, watching movies, and listening to music. 
Based on your diverse needs, I would say that you would need a laptop with high multitasking capability. Do you agree with this assessment?


In [14]:
user_input1 = "budget around 120000 INR"

In [15]:
check_conversation.append({"role":"user", "content":user_input1})

In [16]:
check_assistent_response1 = get_chat_completion(check_conversation)
print(check_assistent_response1)

Great! As a user who needs a laptop for coding, gaming, watching movies, and listening to music, it seems like you require a laptop with a balance of performance and entertainment features.

Based on your requirements, I would suggest the following profile for the laptop:
- GPU intensity: high
- Display quality: high
- Portability: medium
- Multitasking: high
- Processing speed: high

Do you agree with this profile for the laptop?


## Step - 2: Moderation check

In [17]:
def moderation_check(msg):
    response = openai.moderations.create(input=msg)
    output =  response.results[0].flagged
    if output == True:
        return "Flagged"
    else:
        return "Not flagged"

In [18]:
moderation_res = moderation_check("I will kill you.")
print(moderation_res)

Flagged


In [19]:
moderation_res = moderation_check("I will hack NASA.")
print(moderation_res)

Not flagged


In [20]:
moderation_res = moderation_check(check_assistent_response1)
print(moderation_res)

Not flagged


## Step-3: Intent confirmation 

In [21]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt =f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. you need to evaluate if the input has the following keys: 'GPU intensity', 'Display quality', 'portability', 'Multitasking', 'Processing speed'.
    Next you need to evaluate if the keys have the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium' or 'high' based on the importance as stated by user.
    Output a string 'Yes' if the input contains the dictionary with the correctly filled for all keys.
    Otherwise out the string 'No'
    
    Here is the input : {response_assistant}
    Only output a one word string - Yes/no.
    """
    
    
    response = openai.completions.create(
                        model="gpt-3.5-turbo-instruct",
                        prompt = prompt,
                        temperature = 0   
    )
    return response.choices[0].text

In [22]:
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low'}}
"""

In [23]:
print(debug_response_assistant_1)


Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low'}



In [24]:
intent_confirm = intent_confirmation_layer(debug_response_assistant_1)
print(intent_confirm)


Yes


In [25]:
debug_response_assistant_2 = f"""
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'high',
 'Processing speed':'high',
 'Budget':'150000'
 """

In [26]:
intent_confirm = intent_confirmation_layer(debug_response_assistant_2)
print(intent_confirm)


Yes


## Step-3: Dictionary present

In [27]:
def dictionary_present(message):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""
            You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.


            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}


            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}


            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}


            Here is the input {message}
            """
    
    
    response = openai.completions.create(
                        model = "gpt-3.5-turbo-instruct",
                        prompt = prompt,
                        temperature = 0,
                        max_tokens = 900
    )
    return response.choices[0].text

In [28]:
debug_response_assistant_3 = """
{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000'}
"""

In [29]:
response_dictionary = dictionary_present(debug_response_assistant_3)
print(response_dictionary)


{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'low', 'Processing speed': 'low', 'Budget': '50000'}


In [30]:
debug_response_assistant_4 = f"""Thank you for providing your budget.
Based on your budget of 50,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:
- GPU intensity: high
- Display quality: medium
- Portability: low
- Multitasking: high
- Processing speed: high
- Budget: 150,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget.
Let me know if there's anything else I can assist you with!"""

In [31]:
response_dictionary = dictionary_present(debug_response_assistant_4)
print(response_dictionary)


{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '150000'}


In [32]:
debug_conversation = initialize_conversation()
debug_res = get_chat_completion(debug_conversation)
print("Assitent- "+debug_res)
user_input = "I am Abhijit, I want to buy a laptop for coding, seeing movies, listening to music etc."
print("User- "+user_input)
debug_moderation_check = moderation_check(user_input)
debug_conversation.append({"role":"user", "content":user_input})
debug_res1 = get_chat_completion(debug_conversation)
print("Assitent- "+debug_res1)
debug_moderation_check = moderation_check(debug_res1)
debug_conversation.append({"role":"system", "content":debug_res1})
user_input1 = "Budget around 150000 INR"
print("User- "+user_input1)
debug_conversation.append({"role":"user", "content":user_input1})
debug_res2 = get_chat_completion(debug_conversation)
print("Assistent-"+debug_res2)
print("User- "+debug_response_assistant_2)
debug_confirmation = intent_confirmation_layer(debug_response_assistant_2)
print("Assistent- "+debug_confirmation)
print("User- "+debug_response_assistant_2)
response_dictionary = dictionary_present(debug_response_assistant_2)
print("Assistent- "+response_dictionary)

Assitent- Hello! I'm here to help you find the best laptop that suits your needs. Please share your requirements with me so that I can assist you effectively.
User- I am Abhijit, I want to buy a laptop for coding, seeing movies, listening to music etc.
Assitent- Great! It seems like you have a variety of uses for the laptop including coding, watching movies, and listening to music. 
Based on your usage, I would say that you would need a laptop with a good display quality for watching movies, and a decent processing speed for coding. 
Do you have a specific budget in mind for the laptop?
User- Budget around 150000 INR
Assistent-Great! Thank you for providing your budget. 
Based on your usage for coding, watching movies, and listening to music, I would recommend the following values for the laptop features:
- GPU intensity: medium
- Display quality: high
- Portability: medium
- Multitasking: medium
- Processing speed: high

Do you agree with these recommendations?
User- 
{'GPU intensity'

## Step-4: Product map layer

In [33]:
def product_map_layer(laptop_description):
    delimiter = "#####"
    lap_spec = {
      "GPU intensity":"(Type of the Graphics Processor)",
      "Display quality":"(Screen Resolution)",
      "Portability":"(Laptop Weight)",
      "Multitasking":"(RAM)",
      "Processing speed":"(CPU Manufacturer, Core)",
      "Budget":"(Laptop price)"
    }


    values = {'low','medium','high'}


    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec}into {values} based on the following rules: \
    {delimiter}
    GPU Intensity: low: < for entry-level graphics like Intel UHD > , \n
    medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
    high: < High-end dedicated graphics like Nvidia, M2 > , \n


    Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
    medium: < if Full HD resolution (1920x1080) > , \n
    high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n


    Portability: low: < if laptop weight is greater than 2.51 kg > , \n
    medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
    high: < if laptop weight is less than 1.51 kg> \n


    Multitasking: low: < If the value of the RAM is 8GB or lower> , \n
    medium: < if the value of the RAM is between 8GB & 16GB > , \n
    high: < if the value of the RAM is more than 32GB> \n


    Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3 > , \n
    medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5, M1 > , \n
    high: < if High-performance processors like M2, Intel Core i7, AMD Ryzen 7 or higher > \n
    {delimiter}


    {delimiter}
    Here are some input output pairs for few-shot learning:
    input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring basic multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is decently portable. Additionally, it comes with an Intel UHD GPU for basic graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. "
    output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'low','Processing speed':'medium'}}


    input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for productivity. With 16GB of RAM and an SSD, it offers decent multitasking performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for basic graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. "
    output2: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'medium', 'Processing speed':'high'}}


    input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an M2 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an Apple M2 dedicated graphics, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it offers decent portability. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals."
    output3: {{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
    {delimiter}


    Follow the above instructions step-by-step and output the dictionary {lap_spec} for the following laptop {laptop_description}.
    """

    
    response = openai.completions.create(
                    model = 'gpt-3.5-turbo-instruct',
                    prompt = prompt,
                    temperature = 0,
                    max_tokens = 900   
    
    )
    return response.choices[0].text

In [34]:
laptop_description = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""

In [35]:
response_description = product_map_layer(laptop_description)
print(response_description)


{'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium', 'Budget': '35,000'}


In [36]:
type(response_description)

str

In [37]:
df.head(1)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...


In [38]:
df['updated_description'] = df['Description'].apply(lambda x:product_map_layer(x))
df.to_csv('updated_df.csv', index=False, header=True)

In [39]:
new_df = pd.read_csv('updated_df.csv')
new_df.head(2)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,updated_description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"\n{'GPU intensity': 'low', 'Display quality': ..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"\n {'GPU intensity': 'high', 'Display quali..."


In [40]:
new_df['updated_description'][0]

"\n{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'medium', 'Budget': '35,000'}"

## Step-5 :Extract dictionary from string

In [41]:
def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"
    
    dictionary_matches = re.findall(regex_pattern, string)
    if dictionary_matches:
        dictionary_string = dictionary_matches[0].lower()
        
    dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [42]:
dictionary_desc = extract_dictionary_from_string(response_description)
print(dictionary_desc)

{'gpu intensity': 'medium', 'display quality': 'medium', 'portability': 'medium', 'multitasking': 'low', 'processing speed': 'medium', 'budget': '35,000'}


In [43]:
type(dictionary_desc)

dict

## Step-6: Compare laptop with user

In [44]:
def compare_laptop_with_user(user_req_string):
    
#     laptop_df = pd.read_csv('updated_df.csv')
#     user_string_dict = extract_dictionary_from_string(user_string)
#     budget = int(user_string_dict.get('budget', '0').replace(',', '').split()[0])
    
    
#     filtered_laptop = laptop_df.copy()
#     filtered_laptop['Price'] = filtered_laptop['Price'].str.replace(',', '').astype(int)
#     filtered_laptop = filtered_laptop[filtered_laptop['Price'] <= budget].copy()
    
#     #print(filtered_laptop)
#     #return filtered_laptop
#     mappings = {
#         'low': 0,
#         'medium': 1,
#         'high': 2
#     }
    
#     filtered_laptop['Score'] = 0
#     for index, rows in filtered_laptop.iterrows():
#         updated_description = rows['updated_description']
#         laptop_values = extract_dictionary_from_string(updated_description)
#         score=0
        
#         for key, user_val in user_string_dict:
#             if key.lower() == 'budget':
#                 continue
#             laptop_value = laptop_values.get(key, None)
            
#             laptop_mapping = mappings.get(laptop_value.lower(), -1)
#             user_mapping = mappings.get(user_val.lower(), -1)
            
#             if user_mapping >= laptop_mapping:
#                 score+=1
            
#         filtered_laptop.loc[index, 'Score'] = score
        
#     top_laptops = filtered_laptop.drop('updated_description', axis=1)
#     top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    
#     return top_laptops.to_json(orient='records')

    laptop_df= pd.read_csv('updated_df.csv')
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])
    #This line retrieves the value associated with the key 'budget' from the user_requirements dictionary.
    #If the key is not found, the default value '0' is used.
    #The value is then processed to remove commas, split it into a list of strings, and take the first element of the list.
    #Finally, the resulting value is converted to an integer and assigned to the variable budget.




    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    #These lines create a copy of the laptop_df DataFrame and assign it to filtered_laptops.
    #They then modify the 'Price' column in filtered_laptops by removing commas and converting the values to integers.
    #Finally, they filter filtered_laptops to include only rows where the 'Price' is less than or equal to the budget.


    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['updated_description']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0


        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1


        filtered_laptops.loc[index, 'Score'] = score


    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('updated_description', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)


    return top_laptops.to_json(orient='records')
    

In [45]:
top_3_laptops = compare_laptop_with_user(response_description)
print(top_3_laptops)

[{"Brand":"Dell","Model Name":"Inspiron","Core":"i5","CPU Manufacturer":"Intel","Clock Speed":"2.4 GHz","RAM Size":"8GB","Storage Type":"SSD","Display Type":"LCD","Display Size":"15.6\"","Graphics Processor":"Intel UHD","Screen Resolution":"1920x1080","OS":"Windows 10","Laptop Weight":"2.5 kg","Special Features":"Backlit Keyboard","Warranty":"1 year","Average Battery Life":"6 hours","Price":35000,"Description":"The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6\" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance 

In [46]:
type(top_3_laptops)

str

## Step-7 : Product validation layer

In [47]:
def laptop_recommendation_layer(laptop_recommendation):    
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1    

In [48]:
final_data = laptop_recommendation_layer(top_3_laptops)
print(final_data)

[{'Brand': 'Dell', 'Model Name': 'Inspiron', 'Core': 'i5', 'CPU Manufacturer': 'Intel', 'Clock Speed': '2.4 GHz', 'RAM Size': '8GB', 'Storage Type': 'SSD', 'Display Type': 'LCD', 'Display Size': '15.6"', 'Graphics Processor': 'Intel UHD', 'Screen Resolution': '1920x1080', 'OS': 'Windows 10', 'Laptop Weight': '2.5 kg', 'Special Features': 'Backlit Keyboard', 'Warranty': '1 year', 'Average Battery Life': '6 hours', 'Price': 35000, 'Description': 'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU 

In [49]:
type(final_data)

list

## Product recommendation layer

In [50]:
def recommendation_validation(laptop_recommendation):
    """
    Validate a list of laptop recommendations based on a score threshold.

    Parameters:
    - laptop_recommendation (str): JSON string containing a list of laptop recommendations,
      each with a 'Score' key representing its rating.

    Returns:
    - list: A filtered list of laptop recommendations where each item has a 'Score' greater than 2.
    """

In [51]:
validated_data = recommendation_validation(top_3_laptops)
display(validated_data,'\n')

None

'\n'

## Step-8 : product recommendation layer

In [52]:
def initialize_conversation_reco(products):
    """
    Initializes a conversation recommendation system for a laptop gadget expert.

    Parameters:
    - products (list): A list of products to be included in the user's profile.

    Returns:
    - conversation (list): A list containing initial system and user messages for the conversation.

    Description:
    This function sets up a conversation recommendation system for an intelligent laptop gadget expert.
    The system message provides guidance on how to respond to user queries based on the product catalog.
    It instructs to summarize each laptop's major specifications and price, starting with the most expensive.
    The user message confirms the list of products included in the user's profile.

    Example:
    >>> products = ['Laptop A', 'Laptop B', 'Laptop C']
    >>> initialize_conv_reco(products)
    [{'role': 'system', 'content': 'You are an intelligent laptop gadget expert and you are tasked with the objective to solve the user queries about any product from the catalogue in the user message. You should keep the user profile in mind while answering the questions.\n\nStart with a brief summary of each laptop in the following format, in decreasing order of price of laptops:\n1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n\n'},
    {'role': 'user', 'content': " These are the user's products: ['Laptop A', 'Laptop B', 'Laptop C']"}]
    """
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    return conversation
    

In [53]:
debug_conv_reco = initialize_conversation_reco(top_3_laptops)
print(debug_conv_reco)

[{'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and you are tasked with the objective to     solve the user queries about any product from the catalogue in the user message     You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:\n    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>\n\n    '}, {'role': 'user', 'content': ' These are the user\'s products: [{"Brand":"Dell","Model Name":"Inspiron","Core":"i5","CPU Manufacturer":"Intel","Clock Speed":"2.4 GHz","RAM Size":"8GB","Storage Type":"SSD","Display Type":"LCD","Display Size":"15.6\\"","Graphics Processor":"Intel UHD","Screen Resolution":"1920x1080","OS":"Windows 10","Laptop Weight":"2.5 kg","Special Features":"Backlit Keyboard","Warranty":"1 year","Average Battery Life":"6

In [54]:
debug_recommendation = get_chat_completion(debug_conv_reco)
print(debug_recommendation + '\n')

1. Dell Inspiron: Intel Core i5 2.4 GHz, 8GB RAM, SSD, 15.6" LCD 1920x1080, Intel UHD GPU, Windows 10, 2.5 kg, Backlit Keyboard, 6 hours battery life, Rs 35,000
2. HP Pavilion: Intel Core i5 2.3 GHz, 12GB RAM, HDD, 15.6" LCD 1366x768, Intel UHD GPU, Windows 10, 2.1 kg, B&O Audio, 4 hours battery life, Rs 30,000
3. Lenovo IdeaPad: Intel Core i3 2.1 GHz, 8GB RAM, HDD, 15.6" TN 1366x768, Intel UHD GPU, Windows 10, 2.2 kg, Dolby Audio, 5 hours battery life, Rs 25,000

How can I assist you further with these laptops?



In [55]:
response_dictionary

"\n{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '150000'}"

In [56]:
debug_conv_reco.append({"role": "user", "content": "This is my user profile" + str(response_dictionary)})
debug_conv_reco.append({"role": "assistant", "content": debug_recommendation})

In [57]:
debug_user_input = "Which is ideal for travel?"

In [58]:
debug_conv_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_completion(debug_conv_reco)
display('\n' + debug_response_asst_reco + '\n')

'\nBased on your user profile where portability is rated as low, the HP Pavilion would be the most suitable option for travel among the laptops mentioned. It is lightweight at 2.1 kg and offers a good balance between performance and portability.\n'

## Dialouge management system

In [59]:
def dialogue_mgmt_system():
    """
    Manages a conversational system between a user and an assistant to recommend laptops.

    This function initializes the conversation, retrieves the introduction, handles user input,
    performs moderation checks, and manages the recommendation process based on user interactions.

    Returns:
        None
    """

    conversation = initialize_conversation()

    introduction = get_chat_completion(conversation)

    display(introduction + '\n')

    top_3_laptops = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completion(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print(response_assistant)
            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation)

            if confirmation == 'No':
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')

                response = dictionary_present(response_assistant)
                print(response)
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptop_with_user(response)

                print("top 3 laptops are", top_3_laptops)

                validated_reco = recommendation_validation(top_3_laptops)

                conversation_reco = initialize_conversation_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completion(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completion(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [60]:
dialogue_mgmt_system()

"Hello! I'm here to help you find the best laptop that suits your needs. Please share your requirements with me.\n"

 Hi I am abhijit, I am a coder, i need a laptop {'GPU intensity':'high',  'Display quality':'high',  'Portability':'low',  'Multitasking':'low',  'Processing speed':'low',  'Budget':'50000'}


Great to have you here, Abhijit! As a coder, you likely require a laptop with high GPU intensity and display quality for coding and programming tasks. Since you mentioned that multitasking is not a high priority for you, we can keep that at a low level. Additionally, you prefer low portability and processing speed.

Could you please let me know your budget for the laptop? This will help me find the best options that fit your requirements.
Intent Confirmation Yes/No: 
Yes

Great to have you here, Abhijit! As a coder, you likely require a laptop with high GPU intensity and display quality for coding and programming tasks. Since you mentioned that multitasking is not a high priority for you, we can keep that at a low level. Additionally, you prefer low portability and processing speed.

Could you please let me know your budget for the laptop? This will help me find the best options that fit your requirements.


Variables extracted!


{'GPU intensity': 'high', 'Display quality': 'high', 'P

ValueError: invalid literal for int() with base 10: 'unknown'

## Assignment

In [88]:
def extract_details_layer(description):
    delimiter = "#####"
    hotel_spec = {
      "Location":"(Type of the hotel location)",
      "Price":"(Price per night)",
      "Amenities":"(Amenities available)",
      "Reviews and Ratings":"(Review and ratings)",
      "Room Type and Size":"(Room type and size)"
    }


    values = {'best', 'good', 'not good', 'low','medium','high'}


    prompt=f"""
    You are a Hotel Specifications Classifier whose job is to extract the key features of hotels as per their requirements.
    To analyze each hotel, perform the following steps:
    Step 1: Extract the hotels's primary features from the description {description}
    Step 2: Store the extracted features in {hotel_spec} \
    Step 3: Classify each of the items in {hotel_spec}into {values} based on the following rules: \
    {delimiter}
    Location: best: < if Close to hiking trails or if stunning mountain views or if Close bo beach. > , \n
    good: < if distance 5 to 10 minutes > , \n
    not good: < if distance greater than 30 minutes > , \n


    Price: low: < if price less than 5000 > , \n
    medium: < if price greater than 5000 > , \n
    high: < if price greater than 10000 > \n


    Amenities: best: < Wi-Fi, air conditioning, parking, indoor pool, gym, spa, mini-bar, safe, flat-screen TV, toiletries. > , \n
    good: < Wi-Fi, air conditioning, mini-bar, safe, flat-screen TV, toiletries > , \n
    not good: < Wi-Fi, toiletries> \n


    Reviews and Ratings: best: < review greater tha 4 > , \n
    good: < review greater tyhan 3 and less than 4 > , \n
    not good: < review less than 3> \n


    Room Type and Size: best: < Standard, deluxe, and suites, spacious rooms with rustic decor and mountain views or sea beach view. > , \n
    good: < Standard, deluxe, and suites, spacious rooms without decor > , \n
    not good: < spacious rooms without decor > \n
    {delimiter}


    {delimiter}
    Here are some input output pairs for few-shot learning:
    input1: ""Location": "Close to beach",
        "Price": "6000",
        "Amenities": "Wi-Fi, air conditioning, parking, indoor pool, gym, spa, mini-bar, safe, flat-screen TV, toiletries.",
        "Reviews and Ratings": "4.6 stars, praised for cleanliness, service quality, and scenic views.",
        "Room Type and Size": "Standard, deluxe, and suites, spacious rooms with rustic decor and mountain views.","
    output1: {{'Location': 'best','Price':'medium','Amenities':'best','Reviews and Ratings':'best','Room Type and Size':'best'}}


    
    {delimiter}


    Follow the above instructions step-by-step and output the dictionary {hotel_spec} for the following laptop {laptop_description}.
    """

    
    response = openai.completions.create(
                    model = 'gpt-3.5-turbo-instruct',
                    prompt = prompt,
                    temperature = 0,
                    max_tokens = 900   
    
    )
    return response.choices[0].text

In [89]:
des =  f"""Location: Close to beach., Price: 12000 per night,Amenities: Wi-Fi, flat-screen TV, toiletries., Reviews and Ratings": "3.6 stars, praised for cleanliness, service quality, and scenic views., Room Type and Size": "spacious rooms with rustic decor."""

In [90]:
det = extract_details_layer(des)
print(det)


{'Location': 'not good', 'Price': 'low', 'Amenities': 'good', 'Reviews and Ratings': 'good', 'Room Type and Size': 'good'}
